In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_csv('../data_clean/merged_total.csv')
df.fillna('unknown', inplace=True)
df.shape

(250666, 19)

### Embedding

In [16]:
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\storm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\storm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:

embedding_model = "text-embedding-ada-002"

cat_cols = df_clean.select_dtypes(include=['object']).columns
cat_cols = cat_cols[(cat_cols != 'visit_bounce') & (cat_cols != 'contact_contactpersoon_id') & (cat_cols != 'account_account_id')]

cat_cols

Index(['afspraak_keyphrases', 'account_keyphrases', 'campagne_keyphrases',
       'sessie_keyphrases', 'visit_keyphrases', 'mailing_keyphrases'],
      dtype='object')

In [5]:
df[cat_cols].head()

,afspraak_keyphrases,account_keyphrases,campagne_keyphrases,sessie_keyphrases,visit_keyphrases,mailing_keyphrases
0,",","familiebedrijf, diensten, vastgoed, melle gent...","offline, netwerkevenement, ov kick off communi...","ma events, netwerking, netwerkactiviteit project",unknown,unknown
1,",","bedrijf, diensten, consultancy, geraardsbergen...","offline, opleiding, corona round tables sales","gr werking, marketing sales, opleidingen",unknown,unknown
2,"retentie lidmaatschap, ledenbezoek 2023, indus...","bedrijf, diensten, milieu, meilegem oudenaarde...","offline, netwerkevenement, nw nieuwjaarsrecept...","nw voka connect gent, netwerking, netwerkactiv...","chrome, windows, ghent, belgium, pro, event su...","jo stamgasten 2023, uitnodiging stamgasten 23 ..."
3,"retentie lidmaatschap, ledenbezoek 2023, indus...","bedrijf, diensten, milieu, meilegem oudenaarde...","offline, netwerkevenement, nw nieuwjaarsrecept...","nw voka connect gent, netwerking, netwerkactiv...","chrome, windows, ghent, belgium, telenet, acti...",nw nieuwjaarsreceptie vlaamse ardennen leiestr...
4,"retentie lidmaatschap, ledenbezoek 2023, indus...","bedrijf, diensten, milieu, meilegem oudenaarde...","offline, netwerkevenement, nw nieuwjaarsrecept...","nw voka connect gent, netwerking, netwerkactiv...","chrome, windows, ghent, belgium, pro, activite...","nieuwsbrief 09052023, nieuws van voka oost vla..."


In [6]:
len(df['afspraak_keyphrases'])

250666

In [7]:
from nltk.stem.snowball import SnowballStemmer
def remove_stopwords(text):
    stop_words_nl = set(stopwords.words('dutch'))
    
    word_tokens = word_tokenize(text, language='dutch')

    result = [x for x in word_tokens if x not in stop_words_nl]

    seperator = ', '
    return seperator.join(result)


def team_name_change(text):
    teams_dict = {
        'jo': ' jong ondernemen ',
        'do': ' duurzaam ondernemen ',
        'in': ' innovatie digitalisering ',
        'io': ' internationaal ondernemen ',
        'ao': ' arbeidsmarkt ',
        'ex': ' expert ',
        'gr': ' groei ',
        'bb': ' belangenbehartiging ',
        'co': ' communicatie ',
        'nw': ' netwerking ',
        'ha': ' haven ',
        'ma': ' match '
    }
    word_tokens = word_tokenize(text, language='dutch')
    # apply dict to list
    result = [teams_dict.get(word, word) for word in word_tokens]
    # join list to string
    cleaned_list = ', '.join(result)
    # tokenize string
    tokenize_list = word_tokenize(cleaned_list, language='dutch')
    # remove comma
    tokenize_list_no_comma = [x for x in tokenize_list if x != ',']
    # join list to string and remove duplicates from list
    return ', '.join(list(set(tokenize_list_no_comma)))


def stemmer(text):
    stemmer = SnowballStemmer(language='dutch')
    stem_sentence=[]
    for word in text.split(','):
        stem_sentence.append(stemmer.stem(word))
    stem_sentence= ', '.join(stem_sentence)
    return stem_sentence



def clean_text(df, cat_cols=cat_cols):

    df_copy = df.copy()

    for col in cat_cols:
        for row in range(len(df_copy)):
            name_change = team_name_change(df_copy[col][row])
            no_stopwords = remove_stopwords(name_change)
            tokenize_list = word_tokenize(no_stopwords, language='dutch')
            tokenize_list = [x for x in tokenize_list if x != ',']
            df_copy.at[row, col] = ', '.join(list(set(tokenize_list)))
            stemmer_list= stemmer(df_copy[col][row])
            df_copy.at[row, col] = stemmer_list
            
    return df_copy


def flatten_vector_columns(df, col):
    # Flatten the 'vector_column' into a matrix (with padding)
    max_vector_length = max(len(vector) for vector in df[col])
    padded_matrix = np.array([vector + [0.0] * (max_vector_length - len(vector)) for vector in df[col]])
    return padded_matrix

duurt 11.5 minuten om df_clean te maken (10cores) 51 min (4cores)

In [8]:
df_clean = clean_text(df=df, cat_cols=cat_cols)

In [9]:
del df

In [10]:
df_clean[cat_cols].head()

,afspraak_keyphrases,account_keyphrases,campagne_keyphrases,sessie_keyphrases,visit_keyphrases,mailing_keyphrases
0,,"mell, vastgoed, medewerker, gent, dienst, ...","kick, ov, vastgoed, offlin, bouw, off, n...","event, match, netwerk, netwerkactiviteit, ...",unknown,unknown
1,,"consultancy, aalst, bedrijf, dienst, conta...","sales, corona, offlin, tables, round, opleid","werking, sales, market, groei, opleid",unknown,unknown
2,"context, veronique, structur, mens, netwer...","ownermanager, milieu, bedrijf, meilegem, d...","nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...","belgium, ghent, session, chrom, mail, eve...","2023, jong, uitnod, mei, 23, stamgast, o..."
3,"context, veronique, structur, mens, netwer...","ownermanager, milieu, bedrijf, meilegem, d...","nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...","telenet, vlam, nieuwjaarsreceptie, ghent, ...","2023, vlam, leiestrek, nieuwjaarsreceptie, ..."
4,"context, veronique, structur, mens, netwer...","ownermanager, milieu, bedrijf, meilegem, d...","nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...","financiel, ghent, activiteit, chrom, mail,...","digitaliser, rechtstrek, nieuwsbrief, nieuw..."


In [19]:
for col in cat_cols:
    df_clean[col] = df_clean[col].str.replace(r'\d', '', regex=True).str.replace(', ,', ',').str.replace(r'(\s{2},\s{2}),*+', '', regex=True) \
        .str.replace('  ', ' ')
    df_clean[col] = df_clean[col].apply(lambda x: 'unknown' if len(x) == 0 else x)

In [20]:
df_clean[cat_cols].head()

,afspraak_keyphrases,account_keyphrases,campagne_keyphrases,sessie_keyphrases,visit_keyphrases,mailing_keyphrases
0,unknown,"mell, vastgoed, medewerker, gent, dienst, fami...","kick, ov, vastgoed, offlin, bouw, off, netwerk...","event, match, netwerk, netwerkactiviteit, project",unknown,unknown
1,unknown,"consultancy, aalst, bedrijf, dienst, contact, ...","sales, corona, offlin, tables, round, opleid","werking, sales, market, groei, opleid",unknown,unknown
2,"context, veronique, structur, mens, netwerk, l...","ownermanager, milieu, bedrijf, meilegem, diens...","nieuwjaarsreceptie, vlaander, offlin, oost, ne...","provinciaal, gent, netwerk, netwerkactiviteit,...","belgium, ghent, session, chrom, mail, event, s...",", jong, uitnod, mei, stamgast, ondernem"
3,"context, veronique, structur, mens, netwerk, l...","ownermanager, milieu, bedrijf, meilegem, diens...","nieuwjaarsreceptie, vlaander, offlin, oost, ne...","provinciaal, gent, netwerk, netwerkactiviteit,...","telenet, vlam, nieuwjaarsreceptie, ghent, acti...",", vlam, leiestrek, nieuwjaarsreceptie, vlaande..."
4,"context, veronique, structur, mens, netwerk, l...","ownermanager, milieu, bedrijf, meilegem, diens...","nieuwjaarsreceptie, vlaander, offlin, oost, ne...","provinciaal, gent, netwerk, netwerkactiviteit,...","financiel, ghent, activiteit, chrom, mail, man...","digitaliser, rechtstrek, nieuwsbrief, nieuw, v..."


In [21]:
if os.path.exists('../data_clean/rec_system.csv'):
    os.remove('../data_clean/rec_system.csv')
df_clean.to_csv('../data_clean/rec_system.csv', index=False)

In [12]:
df_clean['campagne_keyphrases'][2]

'nieuwjaarsreceptie,  vlaander,  offlin,  oost,  netwerk,  netwerkevenement,  regio'

### embedding

In [13]:
df_clean= pd.read_csv('../data_clean/rec_system.csv')
df_clean.head()

,financieledata_toegevoegde_waarde,financieledata_boekjaar_range,afspraak_keyphrases,afspraak_betreft,afspraak_account_gelinkt,account_keyphrases,inschrijving_aanwezig_afwezig,inschrijving_bron,inschrijving_facturatie_bedrag,campagne_keyphrases,sessie_keyphrases,visit_bounce,visit_duration,visit_first_visit,visit_total_pages,visit_keyphrases,visit_bouce,sentemail_kliks_clicks,mailing_keyphrases
0,413465.00,5.0,unknown,-1.0,-1.0,"mell, vastgoed, medewerker, gent, dienst, ...",1,1,50.0,"kick, ov, vastgoed, offlin, bouw, off, n...","event, match, netwerk, netwerkactiviteit, ...",unknown,-1.0,-1.0,-1.0,unknown,-1,-1.0,unknown
1,494107.00,9.0,unknown,-1.0,-1.0,"consultancy, aalst, bedrijf, dienst, conta...",1,1,0.0,"sales, corona, offlin, tables, round, opleid","werking, sales, market, groei, opleid",unknown,-1.0,-1.0,-1.0,unknown,-1,-1.0,unknown
2,1687010.57,3.0,"context, veronique, structur, mens, netwer...",2.0,1.0,"ownermanager, milieu, bedrijf, meilegem, d...",1,1,0.0,"nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...",Ja,0.0,0.0,1.0,"belgium, ghent, session, chrom, mail, eve...",1,52.0,", jong, uitnod, mei, stamgast, ondernem"
3,1687010.57,3.0,"context, veronique, structur, mens, netwer...",2.0,1.0,"ownermanager, milieu, bedrijf, meilegem, d...",1,1,0.0,"nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...",Ja,0.0,0.0,1.0,"telenet, vlam, nieuwjaarsreceptie, ghent, ...",1,18.0,", vlam, leiestrek, nieuwjaarsreceptie, vla..."
4,1687010.57,3.0,"context, veronique, structur, mens, netwer...",2.0,1.0,"ownermanager, milieu, bedrijf, meilegem, d...",1,1,0.0,"nieuwjaarsreceptie, vlaander, offlin, oost,...","provinciaal, gent, netwerk, netwerkactivite...",Ja,0.0,0.0,1.0,"financiel, ghent, activiteit, chrom, mail,...",1,136.0,"digitaliser, rechtstrek, nieuwsbrief, nieuw..."


In [22]:
df_clean.shape

(250666, 19)

In [34]:
import pandas as pd
import spacy

# Load the spaCy pre-trained Word2Vec model
nlp_en = spacy.load("en_core_web_md")
nlp_nl = spacy.load("nl_core_news_sm")

# Create a sample DataFrame
sentence = df_clean['campagne_keyphrases'][2]

# Define a function to get the Word2Vec vector for a keyphrase
def embed_sentence(sentence):
    # data= []
    # for word in sentence.split(','):
    #     doc = nlp_nl(word)
    #     data.append(word)
    # print(len(data))
    doc = nlp_nl(sentence)
    return np.mean(doc.vector)

def embedding_model(df_clean, cat_cols=cat_cols):
    df_copy = df_clean.copy()
    for col in cat_cols:
        for row in range(len(df_copy)):
            df_copy.at[row, col] = embed_sentence(df_copy[col][row])
    return df_copy


duurt 157 min (4cores) #rip

In [37]:
df_embed = embedding_model(df_clean=df_clean, cat_cols=cat_cols)

In [38]:
df_embed.head()

,financieledata_toegevoegde_waarde,financieledata_boekjaar_range,afspraak_keyphrases,afspraak_betreft,afspraak_account_gelinkt,account_keyphrases,inschrijving_aanwezig_afwezig,inschrijving_bron,inschrijving_facturatie_bedrag,campagne_keyphrases,sessie_keyphrases,visit_bounce,visit_duration,visit_first_visit,visit_total_pages,visit_keyphrases,visit_bouce,sentemail_kliks_clicks,mailing_keyphrases
0,413465.00,5.0,0.013675,-1.0,-1.0,-0.009223,1,1,50.0,-0.008486,-0.006258,unknown,-1.0,-1.0,-1.0,0.013675,-1,-1.0,0.013675
1,494107.00,9.0,0.013675,-1.0,-1.0,-0.003056,1,1,0.0,-0.004275,-0.010647,unknown,-1.0,-1.0,-1.0,0.013675,-1,-1.0,0.013675
2,1687010.57,3.0,-0.007187,2.0,1.0,-0.000462,1,1,0.0,-0.003388,-0.004251,Ja,0.0,0.0,1.0,-0.010881,1,52.0,0.002856
3,1687010.57,3.0,-0.007187,2.0,1.0,-0.000462,1,1,0.0,-0.003388,-0.004251,Ja,0.0,0.0,1.0,-0.011375,1,18.0,-0.004773
4,1687010.57,3.0,-0.007187,2.0,1.0,-0.000462,1,1,0.0,-0.003388,-0.004251,Ja,0.0,0.0,1.0,-0.016026,1,136.0,-0.006436


In [39]:
if os.path.exists('../data_clean/rec_system_embedded.csv'):
    os.remove('../data_clean/rec_system_embedded.csv')
df_embed.to_csv('../data_clean/rec_system_embedded.csv', index=False)

### CLUSTERING

In [40]:
from sklearn.cluster import DBSCAN

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5, ).fit(df_clean)

In [ ]:
labels_db = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_noise_ = list(labels_db).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 60
Estimated number of noise points: 9604


In [ ]:
dbscan.get_params(deep=True)

{'algorithm': 'auto',
 'eps': 0.5,
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'min_samples': 5,
 'n_jobs': None,
 'p': None}